# Mistral-Powered Blog Generator: Code Explanation

This notebook explains the implementation of a Streamlit web application that uses the Mistral-7B language model to generate blog posts. The application allows users to enter a topic, and the model generates a blog post about that topic. Users can also adjust generation parameters and download the generated blog as a Word document.

## 1. Import Required Libraries

Let's break down the libraries being imported:

In [ ]:
# Web application framework
import streamlit as st

# Hugging Face Transformers libraries for the language model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# PyTorch for tensor operations
import torch

# For suppressing warnings
import warnings

# For creating in-memory files
from io import BytesIO

# For creating Word documents
from docx import Document

# Suppress warnings
warnings.filterwarnings("ignore")

## 2. Streamlit Page Configuration

Here we configure the Streamlit page appearance:

In [ ]:
# Configuring the page title, icon, layout, and sidebar state
st.set_page_config(
    page_title="Mistral Blog Generator",
    page_icon="📝",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for styling the app
st.markdown("""
    <style>
    .main .block-container {
        padding-top: 2rem;
    }
    .stButton>button {
        width: 100%;
    }
    </style>
    """, unsafe_allow_html=True)

# Set the app title
st.title("📝 Mistral-Powered Blog Generator")

## 3. Loading the Mistral Model

This function loads the Mistral-7B language model. The `@st.cache_resource` decorator ensures the model is only loaded once and then cached, saving time and resources on subsequent runs:

In [ ]:
@st.cache_resource(show_spinner=True)
def load_model():
    with st.spinner("Loading Mistral-7B-Instruct model. This may take a few minutes..."):
        # Load the tokenizer for Mistral-7B-Instruct
        tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
        
        # Load the model - using float16 precision and automatic device mapping for efficiency
        model = AutoModelForCausalLM.from_pretrained(
            "mistralai/Mistral-7B-Instruct-v0.1",
            torch_dtype=torch.float16,  # Use half-precision to reduce memory usage
            device_map="auto"          # Let the library decide how to map model to available hardware
        )
        
        # Return a text generation pipeline that combines the model and tokenizer
        return pipeline("text-generation", model=model, tokenizer=tokenizer)

## 4. Blog Generation Function

This function takes a topic and generation parameters, then creates a prompt and generates blog content using the Mistral model:

In [ ]:
def generate_blog(topic, max_length=512, temperature=0.7, top_p=0.9):
    # Create a prompt with the Mistral instruction format [INST] ... [/INST]
    prompt = f"[INST] Write a detailed blog post about {topic}. [/INST]"
    
    # Generate the blog post
    result = generator(
        prompt, 
        max_length=max_length,     # Maximum length of the generated text
        do_sample=True,            # Use sampling instead of greedy decoding
        temperature=temperature,   # Controls randomness (higher = more random)
        top_p=top_p                # Nucleus sampling parameter (filters token probabilities)
    )[0]["generated_text"]
    
    return result

## 5. Word Document Creation Function

This function creates a Word document from the generated blog content, allowing users to download their blog post:

In [ ]:
def create_word_doc(content):
    """
    Creates a Word document from the generated blog content.
    """
    # Create a new Word document
    doc = Document()
    
    # Add a title heading
    doc.add_heading('Generated Blog Post', 0)
    
    # Add the blog content as a paragraph
    doc.add_paragraph(content)
    
    # Save the document to a BytesIO object (in-memory file)
    bio = BytesIO()
    doc.save(bio)
    bio.seek(0)  # Reset the file pointer to the beginning
    
    return bio

## 6. Main UI Function

The main function creates the Streamlit user interface with input controls and handles the blog generation workflow:

In [ ]:
def main():
    # Create the sidebar with generation settings
    st.sidebar.title("⚙️ Generation Settings")
    mode = st.sidebar.radio("Mode", ["Simple", "Advanced"])
    
    # Default parameter values
    max_length = 512
    temperature = 0.7
    top_p = 0.9
    
    # Show advanced settings if selected
    if mode == "Advanced":
        max_length = st.sidebar.slider("Maximum Length", min_value=256, max_value=1024, value=512, step=128)
        temperature = st.sidebar.slider("Temperature", min_value=0.1, max_value=1.5, value=0.7, step=0.1)
        top_p = st.sidebar.slider("Top-p", min_value=0.1, max_value=1.0, value=0.9, step=0.1)
    
    # Information about Mistral-7B in the sidebar
    with st.sidebar.expander("ℹ️ About Mistral-7B"):
        st.markdown("**Mistral-7B-Instruct** is a 7 billion parameter language model fine-tuned for following instructions.")
    
    # Main input for blog topic
    topic = st.text_input("Enter a blog topic:", "How to Learn Machine Learning")
    
    # Generate button and handling
    if st.button("Generate Blog", type="primary"):
        if topic.strip():
            with st.spinner("Generating blog... please wait ⏳"):
                # Load model if not already loaded
                if 'generator' not in st.session_state:
                    st.session_state.generator = load_model()
                    
                generator = st.session_state.generator
                # Generate the blog
                blog_content = generate_blog(topic, max_length, temperature, top_p)
                
                # Display the generated blog
                st.subheader(f"🧠 Generated Blog: {topic}")
                st.markdown(blog_content)
                
                # Create a download button for the Word document
                word_doc = create_word_doc(blog_content)
                st.download_button(
                    label="Download Blog Post (Word Format)",
                    data=word_doc,
                    file_name=f"{topic.replace(' ', '_').lower()}_blog.docx",
                    mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
                )
        else:
            st.warning("Please enter a valid topic!")
    
    st.caption("Note: Blog generation may take 10-30 seconds depending on length and hardware.")

## 7. Application Entry Point

Finally, we need to call the main function when the script is run directly:

In [ ]:
if __name__ == "__main__":
    main()

## Key Components and Concepts

### 1. Streamlit Framework
- **Streamlit** is used to create the web interface with minimal code
- Components like text inputs, sliders, buttons, and spinners create an interactive UI
- `st.session_state` maintains the loaded model between reruns

### 2. Hugging Face Transformers
- **AutoTokenizer** converts text to tokens that the model can understand
- **AutoModelForCausalLM** loads the Mistral-7B language model
- **pipeline** simplifies working with the model through a unified interface

### 3. Generation Parameters
- **max_length**: Controls how long the generated blog can be
- **temperature**: Controls randomness/creativity (higher = more random)
- **top_p**: Controls diversity via nucleus sampling

### 4. Performance Optimizations
- Using `torch.float16` reduces memory requirements by using half-precision
- `device_map="auto"` optimizes model loading across available GPUs/CPU
- `@st.cache_resource` prevents reloading the model on each interaction

### 5. Document Generation
- The `python-docx` library creates Word documents
- `BytesIO` creates in-memory files for download without disk storage

## Running the Application

To run this Streamlit app:

1. Install the required packages:
```bash
pip install streamlit transformers torch python-docx
```

2. Save the code to a file (e.g., `blog_generator.py`)

3. Run the Streamlit app:
```bash
streamlit run blog_generator.py
```

4. A browser window will open with the application

**Note**: The Mistral-7B model requires significant resources. You'll need a computer with at least 16GB RAM, and ideally a GPU with 8+ GB VRAM for reasonable performance.